# scanpy in memory computing with BRAD

BRAD's code calling module is capable of running any python script. It uses a standard template for calling each script where the script must accept arguments for:
1. output directory
2. output file (optional)
3. input file (optional)

Suppose we want to do any of scanpy's operations in memory using a single script. This script will take the above 4 arguments in addition to a list, where the list contains a series of scanpy arguments that should run sequentially. The script will always begin with loading a `h5ad` file using scanpy, then execute the list of commands in memory, and finally write the file to the output directory.

Currently, I want to demo this in a jupyter notebook enviornment. Please give me a draft of the main method that will work in the file.

In [8]:
import argparse
import scanpy as sc
import os

def scanpy_main(output_directory, output_file, input_file, scanpy_commands):

    # Load the h5ad file using scanpy
    adata = sc.read_h5ad(input_file)
    print(f'adata loaded from {input_file}')
    print(f'adata.shape={adata.shape}')
    print(f'adata.obs.columns={adata.obs.columns}')
    print(f'adata.obs.head()={adata.obs.head()}')
    print(f'adata.var.head()={adata.var.head()}')

    # Execute the list of scanpy commands in memory
    for command in scanpy_commands:
        print(command)
        exec(command)

    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    output_path = os.path.join(output_directory, output_file)
    adata.write(output_path)


In [9]:
output_directory = "~"
output_file = "processed_data.h5ad"
input_file = "/home/jpic/BRAD-Tools/tests/TS_Heart.h5ad"
scanpy_commands = [
    "sc.pp.filter_cells(adata, min_genes=200)",
    "sc.pp.filter_genes(adata, min_cells=3)",
    "sc.pp.normalize_total(adata, target_sum=1e4)",
    "sc.pp.log1p(adata)",
    "sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)",
    "adata = adata[:, adata.var.highly_variable]",
    "sc.pp.scale(adata, max_value=10)",
    "sc.tl.pca(adata, svd_solver='arpack')",
    "sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)",
    "sc.tl.umap(adata)"
]

scanpy_main(output_directory, output_file, input_file, scanpy_commands)


adata loaded from /home/jpic/BRAD-Tools/tests/TS_Heart.h5ad
adata.shape=(11505, 58870)
adata.obs.columns=Index(['organ_tissue', 'method', 'donor', 'anatomical_information',
       'n_counts_UMIs', 'n_genes', 'cell_ontology_class', 'free_annotation',
       'manually_annotated', 'compartment', 'gender'],
      dtype='object')
adata.obs.head()=                                           organ_tissue method  donor  \
cell_id                                                                 
AAACCCAAGAGCAAGA_TSP12_Heart_Atria_10X_1_1        Heart    10X  TSP12   
AAACCCAAGATGGCGT_TSP12_Heart_Atria_10X_1_1        Heart    10X  TSP12   
AAACCCAAGGGTTAAT_TSP12_Heart_Atria_10X_1_1        Heart    10X  TSP12   
AAACCCAAGTATGCAA_TSP12_Heart_Atria_10X_1_1        Heart    10X  TSP12   
AAACCCAAGTCGTTAC_TSP12_Heart_Atria_10X_1_1        Heart    10X  TSP12   

                                           anatomical_information  \
cell_id                                                             
AAACCC